In [7]:
!pip install glouton

In [9]:
import numpy as np
import pandas as pd
from binascii import hexlify,unhexlify
from enum import Enum

In [11]:
# get this from db.satnogs.org
# sign up and then go to your profile at the top right and get api key
api_key = '#######'

In [13]:
from glouton.domain.parameters.programCmd import ProgramCmd
from glouton.services.observation.observationsService import \
ObservationsService
norad_id =43666

In [15]:
help(ProgramCmd)


Help on class ProgramCmd in module glouton.domain.parameters.programCmd:

class ProgramCmd(builtins.object)
 |  ProgramCmd(norad_id, ground_station_id, start_date, end_date, observation_status, working_dir, archives, waterfalls, demoddata, archive_modules, archive_end_modules, demoddata_modules, demoddata_end_modules, waterfall_modules, waterfall_end_modules, user, transmitter_uuid, transmitter_mode, transmitter_type, frame_modules, frame_end_modules, observer, app_source, transmitter)
 |
 |  Methods defined here:
 |
 |  __init__(self, norad_id, ground_station_id, start_date, end_date, observation_status, working_dir, archives, waterfalls, demoddata, archive_modules, archive_end_modules, demoddata_modules, demoddata_end_modules, waterfall_modules, waterfall_end_modules, user, transmitter_uuid, transmitter_mode, transmitter_type, frame_modules, frame_end_modules, observer, app_source, transmitter)
 |      Initialize self.  See help(type(self)) for accurate signature.
 |
 |  ------------

In [14]:
import datetime

In [15]:
start_date =datetime.datetime(2020,1,1)
end_date=datetime.datetime(2020,2,1)

In [26]:
glouton_conf = ProgramCmd(norad_id=norad_id,
                          ground_station_id=None,
                          start_date=start_date,
                          end_date=end_date,
                          observation_status=None,
                          working_dir=".", # You can change this if you don't want the data in your current directory
                          archives=False,
                          waterfalls=False,
                          demoddata=True,
                          archive_modules=None,
                          demoddata_modules=None,
                          waterfall_modules=None,
                          user=None,
                          transmitter_uuid=None,
                          transmitter_mode=None,
                          transmitter_type=None,
                          frame_modules=None,
                          observer=None,
                          app_source=None,
                          transmitter=None)

obs = ObservationsService(glouton_conf)

Demoddata module(s) loading :
No module list found


In [27]:
%%capture
obs.extract();

In [1]:
from pathlib import Path

In [2]:
files = list(Path('./demoddata__01-01-2020T00-00-00__02-01-2020T00-00-00/').glob('*'))

In [6]:
data=np.fromfile(str(files[0]),np.uint8)

### You now have your data. Here is some AX25 deframing code I started

In [11]:
FRAME_TYPES = {'0x3': 'ui_frame',
          '0x13': 'ui_frame',
          '0x0': 'i_frame',
          '0x2': 'i_frame',
          '0x10': 'i_frame',
          '0x12': 'i_frame'}

In [71]:
ctl = hex(data[15]&0x13)

In [73]:
ctl in FRAME_TYPES

True

In [16]:
def extract_ax25(pkt):
    payload = None
    #HEADER
    header = {}
    header['dest_callsign'] = (data[:6]>>1).tobytes().decode('utf-8')
    header['src_callsign']=(data[7:13]>>1).tobytes().decode('utf-8')
    header['dest_ssid']=ssid_mask(data[6])
    header['src_ssid']=ssid_mask(data[13])
    header['ctl'] = hex(data[14]&0x13)
    #PAYLOAD
    if header['ctl'] in FRAME_TYPES:
        if FRAME_TYPES[header['ctl']]=='i_frame':
            payload = pkt[15:]
        else:
            #process data
            payload= pkt[15:]
    else:
        print(header['ctl'])
    return header,payload
    

In [17]:
c,p=extract_ax25(data)

In [18]:
c

{'dest_callsign': 'EU1XX ',
 'src_callsign': 'EU10S ',
 'dest_ssid': 0,
 'src_ssid': 0,
 'ctl': '0x3'}